## 卷积神经网络
+ Author: xiaoran
+ Time: 2019-02-16 PM.
+ Copy: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/convolutional_neural_network

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# 设置超参数
num_epochs = 5
num_classes = 10
batch_size = 64
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/', train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data/', train=False,transform=transforms.ToTensor())

In [4]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# Convolutional neural network(two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = nn.Linear(7*7*32, num_classes)
    
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


In [6]:
model = ConvNet(num_classes).to(device)

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        lables = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, lables)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/938], Loss: 0.0878
Epoch [1/5], Step [200/938], Loss: 0.0602
Epoch [1/5], Step [300/938], Loss: 0.0507
Epoch [1/5], Step [400/938], Loss: 0.1825
Epoch [1/5], Step [500/938], Loss: 0.0695
Epoch [1/5], Step [600/938], Loss: 0.0677
Epoch [1/5], Step [700/938], Loss: 0.2053
Epoch [1/5], Step [800/938], Loss: 0.0319
Epoch [1/5], Step [900/938], Loss: 0.0200
Epoch [2/5], Step [100/938], Loss: 0.0081
Epoch [2/5], Step [200/938], Loss: 0.1634
Epoch [2/5], Step [300/938], Loss: 0.0807
Epoch [2/5], Step [400/938], Loss: 0.1347
Epoch [2/5], Step [500/938], Loss: 0.0129
Epoch [2/5], Step [600/938], Loss: 0.0103
Epoch [2/5], Step [700/938], Loss: 0.0198
Epoch [2/5], Step [800/938], Loss: 0.0369
Epoch [2/5], Step [900/938], Loss: 0.0291
Epoch [3/5], Step [100/938], Loss: 0.0144
Epoch [3/5], Step [200/938], Loss: 0.0082
Epoch [3/5], Step [300/938], Loss: 0.0225
Epoch [3/5], Step [400/938], Loss: 0.0267
Epoch [3/5], Step [500/938], Loss: 0.0266
Epoch [3/5], Step [600/938], Loss:

In [18]:
# Test the model
model.eval() 
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += lables.size(0)
        correct += (predicted == labels).sum().item()
    print ('Test Accuracy of the model on the 10000 test iamges: {}%'.format(100 * correct / total))
    
torch.save(model.state_dict(), 'ConvNetModel.ckpt')

Test Accuracy of the model on the 10000 test iamges: 197.03423566878982%
